In [1]:
!pip install pyspark
!pip install neo4j
!pip install pandas

import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7cb94f520c2eb2b987d8a2eef42494203b68dc71c077405b6a8c65faa125dc6a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=25c6e6368d82b6e8425cd911e9a1fb116edfaf8994227a26319d4d220106bd79
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
Successfully built neo4j


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ejemplo de PySpark en Jupyter Notebook") \
    .getOrCreate()

# Obtener el SparkContext
sc = spark.sparkContext

# Parte 1

In [3]:
#buckets
B = 2

In [4]:
def hash(x):
    return x % B

In [5]:
# función para hacer map al grafo
def map_pdm(x, arista, y):

    x_hash = hash(x)
    y_hash = hash(y)

    l = []
    for i in range(B):
        l.append(((x_hash, y_hash, i), (x, arista, y)))
        l.append(((i, x_hash, y_hash), (x, arista, y)))
        l.append(((y_hash, i, x_hash), (x, arista, y)))

    return set(l)

In [6]:
graph = [(1,11,2),(1,11,3),(2,11,3),(3,11,2),(3,11,4),(4,11,1),(4,11,2),(4,11,3),(4,12,5),(5,12,1),(5,12,2),(5,12,6)]
#graph = [(2,11,2), (1,11,2), (2,11,1)]

Convertimos ``` graph ``` en un RDD



In [7]:
datos_rdd = sc.parallelize(graph)

In [8]:
datos_rdd.take(5)

[(1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4)]

### Fase de Map

Se ejecuta la función ``` map_pdm()``` obtenido los elementos de los iterables generados.



In [9]:
rdd_map = datos_rdd.flatMap(lambda dato: map_pdm(*dato))

rdd_map.collect()[0:2]

[((1, 0, 0), (1, 11, 2)), ((0, 1, 1), (1, 11, 2))]

### Reduce


Agrupamos los elementos de ``` rdd_map ```



In [10]:
rdd_reduce = rdd_map.groupByKey().mapValues(list)
rdd_reduce.collect()[0]

((1, 0, 0),
 [(1, 11, 2),
  (2, 11, 3),
  (3, 11, 2),
  (3, 11, 4),
  (4, 11, 1),
  (4, 11, 2),
  (4, 11, 3),
  (4, 12, 5),
  (5, 12, 2),
  (5, 12, 6)])

In [11]:
#toma una lista de nodos y detecta todos los triángulos posibles en esa lista.
def triangulo(nodos):
  triangulos_detectados = []
  for i in range(len(nodos)):
    nodo_actual = nodos[i]

    for j in range(i + 1, len(nodos)):

      nodo_sig = nodos[j]

      for k in range(j + 1, len(nodos)):
        nodo_sub_sig = nodos[k]

        if nodo_actual[2] == nodo_sig[0] and nodo_sig[2] == nodo_sub_sig[0] and nodo_actual[0] == nodo_sub_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

        elif nodo_actual[2] == nodo_sub_sig[0] and nodo_sig[0] == nodo_sub_sig[2] and nodo_actual[0] == nodo_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

        elif (nodo_actual[0] == nodo_sub_sig[0] and nodo_actual[2] == nodo_sig[2] and nodo_sub_sig[2] == nodo_sig[0]):
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

  return triangulos_detectados

In [12]:
# Aplicamos la función triangulo a cada elemento del RDD

final = rdd_reduce.map(lambda nodos: (nodos[0], triangulo(nodos[1])))
final.collect()

[((1, 0, 0), [(2, 3, 4)]),
 ((0, 1, 0), [(2, 3, 4)]),
 ((0, 0, 1), [(2, 3, 4)]),
 ((1, 1, 1), []),
 ((0, 1, 1), [(1, 3, 4)]),
 ((1, 0, 1), [(1, 3, 4)]),
 ((1, 1, 0), [(1, 3, 4)]),
 ((0, 0, 0), [])]

In [13]:
dicc = {}
for i in final.collect():
    dicc[i[0]] = i[1]

In [14]:
dicc

{(1, 0, 0): [(2, 3, 4)],
 (0, 1, 0): [(2, 3, 4)],
 (0, 0, 1): [(2, 3, 4)],
 (1, 1, 1): [],
 (0, 1, 1): [(1, 3, 4)],
 (1, 0, 1): [(1, 3, 4)],
 (1, 1, 0): [(1, 3, 4)],
 (0, 0, 0): []}